Diese Version ist die unaufbereitete Version, und verwendet noch Funktionen, welche vor Abgabe verändert werden müssen!

Angepasst werden muss:
<br>[x] nogo_columns
<br>[x] rain id muss raus und an andere stelle
<br>[x] Boxenstopberechnung muss raus
<br>[x] Zielspalte muss umbenannt werden --> irgendwas mit Minuten
<br>[ ] Hyperparameteroptimierung muss fertiggestellt werden
<br>[ ] finales Testing muss rein oder in ein separates Notebook
<br>[ ]  Optimierung des p von Dropout Layer?
<br>[x] nicht nur Vorhersagen( Kommazahlen) vergleichen, sondern direkt sortieren und Podiums Positionen als klare Klassen übergeben

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import random
import operator
import pandas as pd
import numpy as np
import pprint
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
import pandasql as sqldf

Ziel dieses Notebooks ist es Klassen bereit zu stellen, welche anhand einer Grid Search orientierten Methode die Hyperparameter eines Neuronalen Netzes optimieren. 

In der ersten Klasse werden die Hyperparameter Layeranzahl und Neuronenanzahl betrachtet, während die zweite Klasse die Anzahl von Trainingsepochen und die Lernrate für den Adam Optimizer optimiert.

Für die Optimierung wird zusätzlich eine Klasse benötigt, welche dynamisch Neuronale Netze erzeugt.

In [2]:
def load_data(directory = 'sliced_data'):
    '''
        Funktion, die die aufbereiteten/vorbereiteten Daten aus existierenden CSV Dateien 
        einliest und je nach Vollständigkeit in zwei Dictionaries abspeichert,
        geschlüsselt nach der jeweiligen RaceId
    '''
    if os.path.exists(directory):
        csv_filenames = []
        #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
        for filename in os.listdir(os.getcwd()+'/'+directory):
            typ = filename.split('.')[-1]
            name = filename.split('.')[0]
            if typ == 'csv':
                csv_filenames.append(filename)
        sliced_races = {}
        #einlesen und abspeichern als dataframe aller dateien
        for file in csv_filenames:
            try:
                df = pd.read_csv(directory+'/'+file, engine = 'python', sep = ';', decimal = '.')
                del df['Unnamed: 0']
            except Exception as e:
                df = pd.read_csv(directory+'/'+file, engine = 'c', sep = ';', decimal = '.')
                del df['Unnamed: 0']
                print(e)
            f = int(file.split('_')[-1].split('.')[0]) #raceid wird als key gesetzt
            sliced_races[f] = df
        print('Einlesen der '+directory+' Dateien erfolgreich')
    else:
        raise ('sliced Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
   
    return sliced_races

def train_dev_test(data_dict, train_p = 0.7, dev_p = 0.2, test_p = 0.1, nogo_columns = []):
    
    if round(train_p+dev_p+test_p,1) !=1.0:
        raise ValueError ('No valid train/dev/test distribution')
    
    '''
        Daten werde in einem Dictionary übergeben, Dataframes werden in dieser 
        Funktion geshuffled und dann in einen Traindatensatz, einen Development-
        datensatz und in einen Testdatensatz aufgeteilt.
    '''
    #aufteilen in train, dev, test counter
    train_count = round(len(data_dict.keys())*train_p, 0)
    dev_count = train_count+round(len(data_dict.keys())*dev_p,0)
    test_count = len(data_dict.keys())-(train_count+dev_count)
    
    #shufflen der übergebenen Daten
    keys = list(data_dict.keys())
    random.shuffle(keys)
    data_shuffled = {}
    for key in keys:
        data_shuffled[key] = data_dict[key]
        
    #erzeugen separater train,dev,test dictionaries
    train = {}
    dev = {}
    test = {}
    c = 0
    
    #daten sollen nicht in tensoren umgewandelt werden
    for id, df in data_shuffled.items():
        #entfernen nicht gewollter spalten aus dataframe
        cols = [col for col in df.columns if col not in nogo_columns]
        df = df[cols]
        if c < train_count:
            train[id] = df
        elif c >= train_count and c < dev_count:
            dev[id] = df
        else:
            test[id] = df
        c += 1
                
    return train, dev, test

def to_tensor(train_data, dev_data, test_data, nogo_columns = []):
    '''
        Funktion erhält zuvor erzeugte train, dev und test Dictionarys mit Formel 1 Rennen
        und wandelt die DateFrames in Tensoren um. Den Tensoren wird jeweils ihr Target Value
        (podium_podition) zugeordnet. In Dictionary Form werden die Tensoren zurück gegeben.
        Nicht gewollte Spalten (in nogo_columns) werden entfernt.
    '''
    train = []
    train_ = {}
    dev = []
    dev_ = {}
    test = []
    test_ = {}
    
    for id, race in train_data.items():
        for did in race.driverId.unique():
            temp = race.where(race.driverId == did).dropna(how = "all")
            temp_y = list(temp["podium_position"])
            #nicht gewollte Attribute werden aus Datensatz entfernt (bspw. podium_position)
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            #die letze runde eines Fahrers wird betrachtet (enthält kumulierte Informationen über zuvor gefahrene Runden)
            temp_x = temp_x.tail(1)
            x_tensor = torch.tensor(temp_x[cols].values)
            train.append((x_tensor, [temp_y[0]]))
        train_[id] = train
        train = []
    for id, race in dev_data.items():
        for did in race.driverId.unique():
            temp = race.where(race.driverId == did).dropna(how = "all")
            temp_y = list(temp["podium_position"])
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            temp_x = temp_x.tail(1)
            x_tensor = torch.tensor(temp_x[temp_x.columns].values)
            dev.append((x_tensor, [temp_y[0]]))
        dev_[id] = dev
        dev = []
    for id, race in test_data.items():
        for did in race.driverId.unique():
            temp = race.where(race.driverId == did).dropna(how = "all")
            temp_y = list(temp["podium_position"])
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            temp_x = temp_x.tail(1)
            x_tensor = torch.tensor(temp_x[temp_x.columns].values)
            test.append((x_tensor, [temp_y[0]]))
        test_[id] = test
        test = []
        
    return train_,dev_,test_

Einlesen der Dateien als Dataframes, welche in zwei Dictionaries gespeichert werden:<br>
- Attribut directory verweißt auf den Ordnernamen der sliced Dateien, Default ist 'sliced_data'
- Rennen sind standardmäßig nach 50% gesliced (siehe hierzu Datenaufbereitung)

<br> Einmal für das ganze Notebook wird festgelegt, ob Cuda beim Training der Neuronalen Netze verwendet werden soll:

In [3]:
cuda = input('Cuda? [y/n]: ')
sliced_races = load_data(directory = 'sliced_data')

Cuda? [y/n]: y
Einlesen der sliced_data Dateien erfolgreich


Aufteilen des Datensatzes in einen Trainings-, einen Development- und einen Testteil mit Hilfe der Funktion train_dev_test(). Standardmäßig wird der Datensatz zu 70% in Trainings-, zu 20% in Dev- und zu 10% in Testdaten aufgeteilt. Die Verteilung kann mit den Parametern train_p,dev_p,test_p die der Funktion train_dev_test() übergeben werden können, angepasst werden. Wichtig ist bloß, dass insgesamt train_p,dev_p und test_p 1 ergeben. Die Datenbeispiele aus sliced_races werden vor dem Aufteilen geshuffled. Die Funktion train_dev_test() gibt die Datenbeispiele dann wieder in Dictionary Form zurück, wobei jeder RaceId ein DataFrame mit Renndaten zugeordnet wird. Im nächsten Schritt werden die drei Datensammlungen der Funktion to_tensor() übergeben, die nicht gewünschte Spalten aus den Datensätzen entfernt (nogo_columns) und diese dann in Tensoren abspeichert, die wiederrum  ihrem Targetvalue (podiums_position) zugeordnet werden, um die Überprüfung der Vorhersagen später zu vereinfachen. Tupel mit Datensatz und Targetvalue werden dann wieder in dem jeweiligen Dictionary abgespeichert und ihrer RaceId zugeordnet.

In [4]:
#definieren nicht gewollter Attribute (Attribute die nicht dem Modell übergeben werden sollen):
nogo_columns = ['year', 
                'podium_position', 
                'raceId',
                'lap_number',
                'driverId', 
                'driver_fullname',
                'total_milliseconds',
               'lap_in_milliseconds',
               'total_minutes']
#train dev test splitting
train, dev, test = train_dev_test(sliced_races)
#umwandeln in Tensoren und entfernen der nogo_columns
train_T, dev_T, test_T = to_tensor(train, dev, test, nogo_columns)

print('Train:',len(train))
print('Dev:',len(dev))
print('Test:',len(test))
print('Rennen insgesamt:', len(train)+len(dev)+len(test))
print(25*'=')
print('Tensoren:')
print('Train:',len(train_T))
print('Dev:',len(dev_T))
print('Test:',len(test_T))
print('Rennen insgesamt:', len(train_T)+len(dev_T)+len(test_T))

Train: 96
Dev: 27
Test: 14
Rennen insgesamt: 137
Tensoren:
Train: 96
Dev: 27
Test: 14
Rennen insgesamt: 137


#### Definieren von notwendigen Klassen: 
    
Zuerst werden zwei Klassen für Neuronale Netze definiert. Die erste Klasse definiert ein statisches Neuronales Netz, von dem wir wissen, dass es auf unseren Daten gut funktioniert. Die zweite Klasse definiert ein Dynamisches Neuronales Netz, welches in der Hyperparamteroptimierung verwendet wird, um Netze zu erzeugen, die gegen das statische Netz getestet werden.
Der dynamischen Klasse werden Dictionarys übergeben, die spezifizieren wie das Neuronale Netz erzeugt werden soll. Ein Beispiel kann wie folgt aussehen:<br>

{
                 'first'    :    ['linear', 10,150],<br>
                 'relu0'    :    ['linear',150,180],<br>
                 'relu1'    :    ['dropout',0,0],<br>
                 'no_activation0'    :    ['linear',180,190],<br>
                 'relu2'    :    ['linear',190,120],<br>
                 'relu3'    :    ['linear',120,100],<br>
                 'relu4'    :    ['linear',100,70],<br>
                 'relu5'    :    ['linear',70,30],<br>
                 'relu6'    :    ['linear',30,1]
}<br>
         
Dieses Dictionary würde ein Netz erzeugen, wie es in der statischen Klasse definiert wird. Die Keys des Dictionarys geben die dem Layer vorhergehende Activationfunction an (keine für das erste Layer, aus diesem Grund das Stichwort first), während der zugeordnete Value eine Liste mit Layerspezifikationen enthält. An erster Stelle wird der Typ des Layers definiert (dropout oder linear), an zweiter Stelle die Anzahl der Input Neuronen und an dritter Stelle die Anzahl der Output Neuronen für das Layer.

Die nächsten Klassen werden für die Hyperparameteroptimierung definiert. In der ersten Klasse werden dynamisch Neuronale Netze erzeugt, die mit unterschiedlichen Layeranzahlen und Neuronenanzahlen erzeugt werden. Von jeder Layeranzahl wird eine Variation verschiedener Netze erzeugt, sodass eine feste Anzahl Netze mit einer festen Layeranzahl gegen Netz mit einer anderen Anzahl von Layern getestet werden kann. <br>
In der zweiten Klasse werden die Parameter Trainigsepochen und Lernrate für den Adam OPtimizer optimiert. Die Klasse erhält das statische Neuronale Netz und das beste der dynamisch erzeugten Neuronalen Netze und optimiert diese Parameter für beide Netze.

In [5]:
class Netz(nn.Module):
    def __init__(self):
        super(Netz,self).__init__()
        #self.fc1 = nn.Linear(53, 150)
        self.fc1 = nn.Linear(52, 150)
        self.fc2 = nn.Linear(150, 180)
        self.fc3 = nn.Linear(180, 190)
        self.fc4 = nn.Linear(190, 120)
        self.fc5 = nn.Linear(120, 100)
        self.fc6 = nn.Linear(100, 70)
        self.fc7 = nn.Linear(70, 30)
        self.fc8 = nn.Linear(30, 1)
        self.dropout = nn.Dropout()
        
        
    def forward(self,x):
        x = self.fc1(x.float())
        x = F.relu(x.float())
        x = self.fc2(x.float())
        x = F.relu(x.float())
        x = self.dropout(x)
        x = self.fc3(x.float())
        x = F.relu(x.float())
        x = self.fc4(x.float())
        x = F.relu(x.float())
        x = self.fc5(x.float())
        x = F.relu(x.float())
        x = self.fc6(x.float())
        x = F.relu(x.float())
        x = self.fc7(x.float())
        x = F.relu(x.float())
        x = self.fc8(x.float())
        return x

class NetzDynamic(nn.Module):
    
    def __init__(self, layer_information):
        super(NetzDynamic,self).__init__()
        self.__layer_information = layer_information
        #definieren von einer Moduleliste, die alle Layer des Netzes enthalten wird
        self.layers = nn.ModuleList()
        
        #erzeugen der in layer_information übergebeben Layer und hinzufügen zu layers Moduleliste
        for specs in self.__layer_information.values():
            type_ = specs[0]
            in_ = specs[1]
            out_ = specs[2]
            if type_ == 'linear':
                self.layers.append(nn.Linear(in_,out_))
            if type_ == 'dropout':
                self.layers.append(nn.Dropout())
        
    def forward(self, x):
        #definieren eines dynamischen forward pass, activation function richtet sich nach dictionary key
        lay_idx = 0
        for activation, specs in self.__layer_information.items():
            if activation.startswith('relu'):
                layer = self.layers[lay_idx] #auswählen des relevanten Layers aus layers ModuleList
                if specs[0] == 'dropout':
                    x = F.relu(x.float())#anwenden der activation function
                    x = layer(x)
                else:
                    x = F.relu(x.float())
                    x = layer(x.float()) 
            if activation.startswith('sigmoid'):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = torch.sigmoid(x.float())
                    x = layer(x)
                else:
                    x = torch.sigmoid(x.float())
                    x = layer(x.float())
                    
            if activation.startswith('tanh'):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = torch.tanh(x.float())
                    x = layer(x)
                else:
                    x = torch.tanh(x.float())
                    x = layer(x.float())
            if activation.startswith("first"):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float())
            if activation.startswith("no_activation"):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float()) 
            lay_idx += 1#updaten der layerid, um forward pass ins nächste layer zu machen
        
        return x

In [28]:
class HP_Layer_Optimizer(object):
    
    def __init__ (self, 
                  layer_range = (2,8), #range mindestanzahl layer, max anzahl layer
                  input_start = 52, #anzahl inputneuronen für das Inputlayer
                  output_last = 1, #anzahl outputneuronen für das letzte layer
                  in_out_range = (30,200), #range in der sich die anzahl der Neuronen für die Hiddenlayer bewegen soll
                  random_activation = False,#boolean, die angibt ob activation functions random gepicked werden sollen
                  types = ['linear', 'dropout'],#layertypen, von denen gewählt wird
                  activations = ['relu', 'sigmoid', 'tanh'],#activation functions
                  activation_dist = [0.5,0.25,0.25],#wenn random_activation == False wird hier die Häufigkeit für jede Activation übergeben, nicht implementiert
                  dropout_num = 1,#anzahl von dropoutlayern pro NN
                  cuda = True,#soll mit cuda gearbeitet werden?
                  lr = 0.0001,#lernrate für adam optimizer
                  max_epochs = 5,#anzahl der Trainingsepochen
                  create_combinations = True, #sollen von der Klasse Netze erzeugt werden, oder sollen diese von außen in die Klasse gegeben werden?
                  create_variations = False,#soll von jeder Layeranzahl mehr als ein Netz erzeugt werden?
                  num_variations = 4, #wenn Variationen erzeugt werden sollen wird eine festgelegte Zahl Netze mit 4 Layer, eine festgelegte Anzahl mit 5 Layern usw. erzeugt
                  pure_activations = True #wenn random_activation auf True gesetzt ist kann hier festgelegt werden, dass zusätzlich zu absolut zufällig kombinierten activationfunctions
                  #auch noch Netze erzeugt werden, die nur einheitliche activation functions haben (pure sind)
                  ):
        
        self.__model = None #model, welches trainiert und getestet wird, wird hier zwischengespeichert
        self.__layer_range = layer_range #min und max anzahl von layern in Tupelform
        self.__input_start = input_start#Inputgröße für Startlayer/Inputlayer
        self.__output_last = output_last#Outputgröße für letztes Layer / Outputlayer
        self.__in_out_range = in_out_range#range in der sich die Anzahl der In- und Outputs für die Hiddenlayer bewegen soll (Tupelform)
        self.__layer_types = types#liste welche Layertypen enthält (bspw. linear oder dropout)
        self.__dropout_number = dropout_num#anzahl von layern, die pro Netz ein dropout Layer sein sollen
        self.__random_activation = random_activation#Boolean, ob die activation zufällig gewählt werden soll
        self.__create_variations = create_variations
        self.__number_of_variations = num_variations
        self.__pure = pure_activations
        
        self.__combination_results = {}#dictionary, die dem schlüssel zu einer NN Kombi einen MAE zuordnet
        self.model_specs_combinations = {}#dictionary, welches die jeweiligen NN Kombinationen enthält (einem Schlüssel zugeordnet)
        self.train_data = None#trainingsdaten, die dem Optimizer übergeben werden für die Modelle
        self.test_data = None#developmentdaten, um die Modelle zu testen und Aussagen über die besten Kombinationen zu treffen
        self.lr = lr#lernrate für Modell OPtimizer (default Adam)
        self.max_epochs = max_epochs#Anzahl der Trainingsepochen
        self.cuda = cuda#Boolean, ob mit cuda gearbeitet werden soll oder nicht
        self.opt_combination = {}#dictionary enthält optimale kombination aus möglichen kombinationen
        
        self.__activations = None#entweder Liste (wenn activations random ausgewählt werden sollen), oder dictionary, 
        #wenn activation functions mit einer bestimmten Häufigkeit verwendet werden sollen
        if self.__random_activation:#random pick von activation functions
            self.__activations = activations
        else:#activation function sollen mit einer gewissen häufigkeit ausgewählt werden
            k = {}
            for a in range(len(types)):#zuordnen einer häufihkeit aus activation_dist liste zu jeder activation function aus types
                act = types[a]
                dist = activation_dist[a]
                k[act] = dist
            self.__activations = k
            
        #aufrufen der Funktion, die anhand der übergebenen Parameter Modellkombinationen erzeugt
        if create_combinations:
            print("in constructor bevor die kombinationen erzeugt werden")
            self.__create_combinations()
        else:
            print('NN Kombinationen müssen in Dictionary Form selbst übergeben werden')
        
    def __train(self, epoch, optimizer):
        '''
            funktion übernimmt das Training von dem in self.__model
            zwischengespeicherten NN. 
            Epoch: Jetzige Epoche in der trainiert wird (für coolen print Befehl wichtig)
            Optimizer: Optimizer mit dem Parameter des NN aus self.__model optimiert werden
        '''
        if self.cuda:
            self.__model.cuda()
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0).cuda()
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    optimizer.zero_grad()
                    out = self.__model(data)
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    ##rint("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    ##   epoch, batch_id *len(data), len(self.train_data),
                    ##00. * batch_id / len(self.train_data), loss.item()))
                    batch_id +=1
        else:
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1)
                    optimizer.zero_grad()
                    out = self.__model(data)
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    #print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    #    epoch, batch_id *len(data), len(self.train_data),
                    #100. * batch_id / len(self.train_data), loss.item()))
                    batch_id +=1
            
    def __test(self):
        '''
            Funktion, die das Testen des Models aus self.__model auf den übergebenen Dev-Daten 
            (self.test_data) übernimmt und den gesamt MAE für das jeweilige Modell berechnet
        '''
        total = 0
        count = 0
        result_dict = {}
        result = pd.DataFrame(columns = ['target','prediction'])
        help_dict = {}
        if self.cuda:
            self.__model.cuda()
            for key in self.test_data.keys():
                help_dict = {}
                for data, target in self.test_data[key]:
                    self.__model.eval()
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    out = self.__model(data).cpu()
                    out = out.detach().numpy()
                    target = target.cpu()
                    target = target.detach().numpy()
                    total += abs(out - target[0][0])
                    help_dict[target[0][0]] = out
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                
                result = result.append(t, sort = True)
                
        else:
            for raceId in self.test_data.keys():
                help_dict = {}
                for data, target in self.test_data[raceId]:
                    self.__model.eval()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1)
                    out = self.__model(data)
                    out = out.detach().numpy()
                    target = target.detach().numpy()
                    total += abs(out - target[0][0])
                    help_dict[target[0][0]] = out
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                result = result.append(t, sort = True)
        return result
    
    def get_all_information(self):
        
        print('All Model Combinations with encoding:\n', self.model_specs_combinations)
        print('Model Results:\n', self.__combination_results)
        print('Optimale Kombination:\n', self.opt_combination)
        
    def validate_combinations(self):
        
        for key, combination in self.model_specs_combinations.items():
            print("kombination, die jetzt trainiert wird", combination)
            self.__model = NetzDynamic(combination)
            optimizer = optim.Adam(self.__model.parameters(), lr = self.lr)
            #trainieren des modells
            for epoch in range(1,self.max_epochs):
                self.__train(epoch, optimizer)  
            #Aufrufen der Testfunktion
            result = self.__test()
            A = result.prediction.tolist()
            y = result.target.tolist()
            mae = MAE(A,y)
            
            
            
            self.__combination_results[key] = mae
            
        #finden der besten kombination nach minimalstem Error (MAE)
        key_min = min(self.__combination_results.keys(), key=(lambda k: self.__combination_results[k]))
        best_combination = self.model_specs_combinations[key_min]
        best_combination['mae'] = self.__combination_results[key_min]
        self.opt_combination = best_combination
        #self.__combination_overview[key] = specifics
        
        
    def __create_combinations(self):
        '''
            Funktion erzeugt Dictionarys, mit NN Modellspezifikationen, die
            später gegeneinander getestet werden sollen. Kombinationen werden
            in dem Dictionary self.model_specs_combinations unter einem 
            Schlüssel abgespeichert
        '''
        
        if self.__create_variations:
            
            if self.__pure:
                '''
                es werden mehr als ein Netz von einer bestimmten Layeranzahl erzeugt:
                wenn die Layeranzahl Range zwischen 7 und 9 liegt, self.__create_variations = True ist und
                die Variable self.__number_of_variations = 4 ist werden 4 Netze mit 7 Layern und 4 Netze
                mit 8 Layern erzeugt. Diese können dann gegeneinander getestet werden und geben eine bessere
                Übersicht über eine gute Layeranzahl. Wenn self.__pure auf True gesetzt wird, wird aus der 
                Liste mit activationfunctions [relu, sigmoid, tanh] als default 'reine' Netze erzeugt, die immmer
                nur eine der Funktionen enthält (Relu als anfang). Danach werden diese netze kopiert, nur dass die 
                Aktivierungsfunktionen überschrieben werden (ReLu wird durch sigmoid und tanh ersetzt). Zuletzt werden 
                die Funktionen wild gemischt.
                '''
                min_layer = self.__layer_range[0]
                max_layer = self.__layer_range[1]
                if min_layer == max_layer:
                    max_layer += 1
                #pure ist true, deswegen wird zu anfang eine der aktivierungsfunktionen ausgewählt, die fürs ganze netz gewählt wird
                act = random.choice(self.__activations)
                for layer in range(min_layer, max_layer): 
                    variation_counter = 0
                    while variation_counter < self.__number_of_variations:
                        variation_counter +=1 
                        dropout_counter = 0
                        act_count = 0
                        specs_dict = {}
                        middle = layer//2
                        for l in range(layer):
                            layer_specs = []
                            key = act+str(act_count)
                            act_count += 1
                            if dropout_counter == self.__dropout_number:
                                #es wurden schon ausreichend dropout layer erzeugt
                                l_ = random.choice([x for x in self.__layer_types if x not in ['dropout']])
                            else:
                                l_ = random.choice(self.__layer_types)
                                if l_ == 'dropout':
                                    dropout_counter +=1
                                    
                            if l == 0:
                                in_ = self.__input_start
                                if l_ == 'dropout':
                                    dropout_counter = dropout_counter-1
                                    l_ = 'linear'
                                range_start = self.__in_out_range[0]
                                range_end = self.__in_out_range[1]
                                out_ = random.randint(range_start, range_end)
                                layer_specs.append(l_)
                                layer_specs.append(in_)
                                layer_specs.append(out_)
                            else:
                                layer_before = specs_dict[list(specs_dict.keys())[-1]]
                                out_alt = specs_dict[list(specs_dict.keys())[-1]][2]
                                l_type = layer_before[0]
                                i_ltype = -2
                                while l_type == 'dropout':#überprüfen ob vorhergegangenes Layer ein dropout layer war
                                    #sobald das vorhergegangene nicht-dropoutlayer gefunden wurde, wird output größe übernommen
                                    layer_before = specs_dict[list(specs_dict.keys())[i_ltype]]
                                    l_type = layer_before[0]
                                    out_alt = specs_dict[list(specs_dict.keys())[i_ltype]][2]
                                    i_ltype = i_ltype -1
                                if l <= middle: #in der ertsen hälfte nimmt output zu
                                    in_ = out_alt
                                    range_start = out_alt
                                    range_end = self.__in_out_range[1]
                                    out_ = random.randint(range_start, range_end)
                                else:#in zweiter hälfte der layer nimmt output wieder ab
                                    in_ = out_alt
                                    range_start = self.__in_out_range[0]
                                    range_end = out_alt
                                    out_ = random.randint(range_start, range_end)
                                layer_specs.append(l_)
                                layer_specs.append(in_)
                                layer_specs.append(out_)
                            #if l == layer-1:
                            #    l_ = 'linear'
                            #    layer_specs = [l_,in_,self.__output_last]
                            #    specs_dict['last'] = layer_specs#layer wird ohne activation gespeichert und als letztes Layer des NN gekennzeichnet
                                
                            if l == 0:
                                specs_dict['first'] = layer_specs
                            else:
                                if specs_dict[list(specs_dict.keys())[-1]][0] == 'dropout':# keine activation function bei layern direkt nach einem dropout layer
                                    key = 'no_activation'+str(act_count)
                                specs_dict[key] = layer_specs
                        key = random.randint(0,10000)
                        while key in list(self.model_specs_combinations.keys()):
                             key = random.randint(0,10000)
                        self.model_specs_combinations[key]= specs_dict   
                        print(specs_dict,'\n')
                        
                used_acts = [act]
                act_count = 0
                netze = list(self.model_specs_combinations.values())
                mixed = []
                if self.__random_activation:
                    p_ = 0
                    for netz in netze:
                        netz_neu = {}
                        for act_alt, layer_spec in netz.items():
                            if act_alt == 'no_activation' or act_alt == 'first':
                                netz_neu[act_alt] = layer_spec
                            else:
                                act_neu = random.choice(self.__activations)
                                pp = act_neu+str(p_)
                                p_ += 1
                                netz_neu[pp]= layer_spec
                        mixed.append(netz_neu)
                for activation in self.__activations:
                    if activation in used_acts:
                        continue
                    else:
                        for netz in netze:#alle bisher erzeugten netze werden betrachtet
                            key = random.randint(0,10000)
                            while key in list(self.model_specs_combinations.keys()):
                                key = random.randint(0,10000)
                            netz_neu = {}
                            for act_alt, layer in netz.items():
                                if act_alt == 'no_activation' or act_alt == 'first':
                                    netz_neu[act_alt] = layer
                                else:
                                    act_ = activation+str(act_count)
                                    act_count += 1
                                    netz_neu[act_] = layer
                            print(netz_neu,'\n')
                            self.model_specs_combinations[key] = netz_neu
                if len(mixed) != 0:
                    for netz in mixed:
                        print("mixed netz",netz)
                        key = random.randint(0,10000)
                        while key in list(self.model_specs_combinations.keys()):
                            key = random.randint(0,10000)
                        self.model_specs_combinations[key] = netz
                print("alle kombinationen nach create combination:", self.model_specs_combinations)
            else:
                raise Exception('please et pure_activations to True, False not implemented')
                    
        else:
            min_layer = self.__layer_range[0]
            max_layer = self.__layer_range[1]
            if min_layer == max_layer:
                max_layer += 1
            for layer in range(min_layer, max_layer): 
                
                dropout_counter = 0
                act_count = 0
                specs_dict = {}
                middle = layer//2
                for l in range(layer):
                    layer_specs = []
                    if self.__random_activation:#random activation pick ist aktiviert
                        act = random.choice(self.__activations)
                        key = act+str(act_count)
                        act_count += 1
                        if dropout_counter == self.__dropout_number:
                            #es wurden schon ausreichend dropout layer erzeugt
                            l_ = random.choice([x for x in self.__layer_types if x not in ['dropout']])
                        else:
                            l_ = random.choice(self.__layer_types)
                            if l_ == 'dropout':
                                dropout_counter +=1
                                
                        if l == 0:
                            in_ = self.__input_start
                            if l_ == 'dropout':
                                dropout_counter = dropout_counter-1
                                l_ = 'linear'
                            range_start = self.__in_out_range[0]
                            range_end = self.__in_out_range[1]
                            out_ = random.randint(range_start, range_end)
                            layer_specs.append(l_)
                            layer_specs.append(in_)
                            layer_specs.append(out_)
                        else:
                            layer_before = specs_dict[list(specs_dict.keys())[-1]]
                            out_alt = specs_dict[list(specs_dict.keys())[-1]][2]
                            l_type = layer_before[0]
                            i_ltype = -2
                            while l_type == 'dropout':#überprüfen ob vorhergegangenes Layer ein dropout layer war
                                #sobald das vorhergegangene nicht-dropoutlayer gefunden wurde, wird output größe übernommen
                                layer_before = specs_dict[list(specs_dict.keys())[i_ltype]]
                                l_type = layer_before[0]
                                out_alt = specs_dict[list(specs_dict.keys())[i_ltype]][2]
                                i_ltype = i_ltype -1
                            if l <= middle: #in der ertsen hälfte nimmt output zu
                                in_ = out_alt
                                range_start = out_alt
                                range_end = self.__in_out_range[1]
                                out_ = random.randint(range_start, range_end)
                            else:#in zweiter hälfte der layer nimmt output wieder ab
                                in_ = out_alt
                                range_start = self.__in_out_range[0]
                                range_end = out_alt
                                out_ = random.randint(range_start, range_end)
                            layer_specs.append(l_)
                            layer_specs.append(in_)
                            layer_specs.append(out_)
                        #if l == layer-1:
                        #    l_ = 'linear'
                        #    layer_specs = [l_,in_,self.__output_last]
                        #    specs_dict['last'] = layer_specs#layer wird ohne activation gespeichert und als letztes Layer des NN gekennzeichnet
                            
                        #elif l == 0:
                        if l == 0:
                            specs_dict['first'] = layer_specs
                        else:
                            #if l_ == 'dropout':#keine activation function bei dropout layern
                            #    key = 'no_activation'+str(act_count)
                            if specs_dict[list(specs_dict.keys())[-1]][0] == 'dropout':# keine activation function bei layern direkt nach einem dropout layer
                                key = 'no_activation'+str(act_count)
                            specs_dict[key] = layer_specs
                key = random.randint(0,10000)
                while key in list(self.model_specs_combinations.keys()):
                     key = random.randint(0,10000)
                self.model_specs_combinations[key]= specs_dict   
                print(specs_dict)

                
class HP_Optimizer(object):
    
    def __init__(self, lr_range = (0.0001,0.0001), step_size = 0.0001, max_epochs = (2,2), opt = 'Adam', cuda = True, dynamic = False, dyn_combination = {}):
        
        self.__model = Netz()
        self.__lr = lr_range
        self.__epochs = max_epochs
        self.__optimizer = opt
        self.__steps = step_size
        self.__combination_results = {}
        self.__combination_overview = {}
        self.train_data = None
        self.test_data = None
        self.cuda = cuda
        self.opt_combination = {}
        self.__dynamic = dynamic
        self.__dyn_combination = dyn_combination
        
        
    def validate_combinations(self):
        
        specifics = {}
        if self.__optimizer == 'Adam':
            
            #definieren der range für die lernratenoptimierung
            '''
            
            IST HIER WAS KAPUTT?
            
            '''
            if isinstance(self.__lr, tuple):
                lr_s = self.__lr[0]
                lr_e = self.__lr[1]
                #wurde eine range für die anzahl der epochen übergeben?
                if self.__epochs[0] == self.__epochs[1]:
                    #hyperparamter epochenanzahl wird nicht optimiert
                    max_epoch = self.__epochs[0]
                    if lr_s == lr_e:
                        #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                        print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                        #setzen des optimizers als Adam und erzeugen des Modells
                        if self.__dynamic:
                            self.__model = NetzDynamic(self.__dyn_combination)
                        else:
                            self.__model = Netz()
                        optimizer = optim.Adam(self.__model.parameters(), lr = lr_s)
                        for epoch in range(1,max_epoch):
                            self.__train(epoch, optimizer)     
                        result = self.__test()
                        A = result.prediction.tolist()
                        y = result.target.tolist()
                        mae = MAE(A,y)
                        specifics = {}
                        specifics['lr'] = lr_s
                        specifics['epochen'] = max_epoch
                        key = random.randint(0,10000)
                        while key in list(self.__combination_results.keys()):
                             key = random.randint(0,10000)
                        self.__combination_results[key] = mae
                        self.__combination_overview[key] = specifics 
                    else:
                        for l in np.arange(lr_s,lr_e, self.__steps):
                            #setzen des optimizers als Adam und erzeugen des Modells
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = l)
                            #trainieren des modells
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)  
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = l
                            specifics['epochen'] = max_epoch
                            #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics
                else:
                    for max_epoch in range(self.__epochs[0], self.__epochs[1]):
                        #definieren der range für die lernratenoptimierung
                        lr_s = self.__lr[0]
                        lr_e = self.__lr[1]
                        #hyperparamter epochenanzahl wird nicht optimiert
                        if lr_s == lr_e:
                            #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                            #print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = lr_s)
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)       
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = lr_s
                            specifics['epochen'] = max_epoch
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)  
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics          
                        else:
                            for l in np.arange(lr_s,lr_e, self.__steps):
                                if self.__dynamic:
                                    self.__model = NetzDynamic(self.__dyn_combination)
                                else:
                                    self.__model = Netz()
                                optimizer = optim.Adam(self.__model.parameters(), lr = l)
                                #trainieren des modells
                                for epoch in range(1,max_epoch):
                                    self.__train(epoch, optimizer)  
                                result = self.__test()
                                A = result.prediction.tolist()
                                y = result.target.tolist()
                                mae = MAE(A,y)
                                specifics = {}
                                specifics['lr'] = l
                                specifics['epochen'] = max_epoch
                                #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                                key = random.randint(0,10000)
                                while key in list(self.__combination_results.keys()):
                                     key = random.randint(0,10000)
                                self.__combination_results[key] = mae
                                self.__combination_overview[key] = specifics
                #finden der besten kombination nach minimalstem Error (MAE)
                key_min = min(self.__combination_results.keys(), key=(lambda k: self.__combination_results[k]))
                best_combination = self.__combination_overview[key_min]
                best_combination['mae'] = self.__combination_results[key_min]
                self.opt_combination = best_combination
            elif isinstance(self.__lr, list):
                #wurde eine range für die anzahl der epochen übergeben?
                if self.__epochs[0] == self.__epochs[1]:
                    #hyperparamter epochenanzahl wird nicht optimiert
                    max_epoch = self.__epochs[0]
                    if len(self.__lr) ==1:
                        #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                        print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                        #setzen des optimizers als Adam und erzeugen des Modells
                        if self.__dynamic:
                            self.__model = NetzDynamic(self.__dyn_combination)
                        else:
                            self.__model = Netz()
                        optimizer = optim.Adam(self.__model.parameters(), lr = self.__lr[0])
                        for epoch in range(1,max_epoch):
                            self.__train(epoch, optimizer)     
                        result = self.__test()
                        A = result.prediction.tolist()
                        y = result.target.tolist()
                        mae = MAE(A,y)
                        specifics = {}
                        specifics['lr'] = self.__lr[0]
                        specifics['epochen'] = max_epoch
                        key = random.randint(0,10000)
                        while key in list(self.__combination_results.keys()):
                             key = random.randint(0,10000)
                        self.__combination_results[key] = mae
                        self.__combination_overview[key] = specifics 
                    else:
                        for l in self.__lr:
                            #setzen des optimizers als Adam und erzeugen des Modells
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = l)
                            #trainieren des modells
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)  
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = l
                            specifics['epochen'] = max_epoch
                            #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics
                else:
                    for max_epoch in range(self.__epochs[0], self.__epochs[1]):
                        #definieren der range für die lernratenoptimierung
                        lr_s = self.__lr[0]
                        lr_e = self.__lr[1]
                        #hyperparamter epochenanzahl wird nicht optimiert
                        if len(self.__lr) == 1:
                            #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                            #print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = self.__lr[0])
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)       
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = self.__lr[0]
                            specifics['epochen'] = max_epoch
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)  
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics          
                        else:
                            for l in self.__lr:
                                if self.__dynamic:
                                    self.__model = NetzDynamic(self.__dyn_combination)
                                else:
                                    self.__model = Netz()
                                optimizer = optim.Adam(self.__model.parameters(), lr = l)
                                #trainieren des modells
                                for epoch in range(1,max_epoch):
                                    self.__train(epoch, optimizer)  
                                result = self.__test()
                                A = result.prediction.tolist()
                                y = result.target.tolist()
                                mae = MAE(A,y)
                                specifics = {}
                                specifics['lr'] = l
                                specifics['epochen'] = max_epoch
                                #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                                key = random.randint(0,10000)
                                while key in list(self.__combination_results.keys()):
                                     key = random.randint(0,10000)
                                self.__combination_results[key] = mae
                                self.__combination_overview[key] = specifics
                #finden der besten kombination nach minimalstem Error (MAE)
                key_min = min(self.__combination_results.keys(), key=(lambda k: self.__combination_results[k]))
                best_combination = self.__combination_overview[key_min]
                best_combination['mae'] = self.__combination_results[key_min]
                self.opt_combination = best_combination
                
        else:
            raise ('No valid optimizer given! Try Adam for example!')
            
    def __train(self, epoch, optimizer):
        if self.cuda:
            self.__model.cuda()
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0).cuda()
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    optimizer.zero_grad()
                    out = self.__model(data)
                    #print("Out: ", out, out.size())
                    #print("Target: ", target, target.size())
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    #rint("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    #   epoch, batch_id *len(data), len(self.train_data),
                    #00. * batch_id / len(self.train_data), loss.item()))
                    batch_id +=1
        else:
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    #data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)#.cuda()
                    shape = target.size()[1]
                    target = target.resize(shape,1)#.cuda()
                    optimizer.zero_grad()
                    out = self.__model(data)
                    #print("Out: ", out, out.size())
                    #print("Target: ", target, target.size())
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    #print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    #    epoch, batch_id *len(data), len(train_data),
                    #100. * batch_id / len(train_data), loss.item()))
                    batch_id +=1
                    
            
    def __test(self):
        total = 0
        count = 0
        result_dict = {}
        result = pd.DataFrame(columns = ['target','prediction'])
        help_dict = {}
        if self.cuda:
            self.__model.cuda()
            for key in self.test_data.keys():
                #print(key)
                help_dict = {}
                for data, target in self.test_data[key]:
                    self.__model.eval()
                    #files.listdir(path)
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    out = self.__model(data).cpu()
                    #print(out)
                    out = out.detach().numpy()
                    #out = np.round(out)
                    target = target.cpu()
                    target = target.detach().numpy()
                    #print(data)
                    #print(data["driverId"])
                    total += abs(out - target[0][0])
                    #print("current_position: ", data[0][0].item())
                    #print("Output: ", out)
                    #print("Target: ", target)
                    help_dict[target[0][0]] = out
                    #print("Difference: ", out - target)
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                
                result = result.append(t, sort = True)
                
        else:
            for raceId in self.test_data.keys():
                #print(key)
                help_dict = {}
                for data, target in self.test_data[raceId]:
                    self.__model.eval()
                    #files.listdir(path)
                    #data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1)#.cuda()
                    out = self.__model(data)#.cpu()
                    #print(out)
                    out = out.detach().numpy()
                    #out = np.round(out)
                    #target = target.cpu()
                    target = target.detach().numpy()
                    #print(data)
                    #print(data["driverId"])
                    total += abs(out - target[0][0])
                    #print("current_position: ", data[0][0].item())
                    #print("Output: ", out)
                    #print("Target: ", target)
                    help_dict[target[0][0]] = out
                    #print("Difference: ", out - target)
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                result = result.append(t, sort = True)
        return result
         
            
    def get_all_information(self):
        
        print('Chosen Model:',self.__model)
        print('Learningrate Range:',self.__lr)
        print('Maximum Epochs:', self.__epochs)
        print('Chosen Optimizer:', self.__optimizer)
        print('Result Encoding:', self.__combination_overview)
        print('Results:', self.__combination_results)
        print('Optimale Kombination:', self.opt_combination)
        
    def help(self):
        print('Parameters with defaults:\nlr_range --> (0.0001,0.0001),\nstep_size--> 0.0001,\nmax_epochs-->(2,2),\nopt-->"Adam",\ncuda=True')
        print('lr_range: Tupel with learnrate range')
        print('step_size: float/int for step_size of learnrate')
        print('max_epochs: Tupel with number of epochs range')
        print('opt: Optimizer (by default Adam)')
        print('cuda: True/False if cuda should be used, default = True\n')
        print('Attributes:')
        print('set self.train_data as dictionary with races (form: {raceId: race(dataframe)})')
        print('set self.test_data as dictionary with races (form: {raceId: race(dataframe)})')
        print('self.opt_combination: Dictionary which contains the best combination of the given parameters\n')
        print('Methods:')
        print('call self.validate_combination() to compare all combinations')
        print('get all information/results with self.get_all_information()')

#### Finden des "optimalsten" Netzes  
Aufrufen des Layeroptimizers, um verschiedene Neuronale Netze gegeneinander zu testen, mit verschiedenen Layeranzahlen, und Input und Output Neuronen. Gemeinsam haben die Netze, dass bis zur Mitte die Anzahl Neuronen zunimmt, und dann zum letzten Layer hin wieder abnimmt.
<br>Es wird das Netz ausgewählt, welches den niedrigsten MAE auf dem Dev Datensatz hat. Ziel ist es durch Aufrufen dieser Klasse die beste Layeranzahl-Aktivierungsfunktionen-Kombination für den Use Case zu finden (anhand vorgegebener einschränkender Paramter). Da die Anzahl der Neuronen jedes Mal zufällig implementiert wird, und dieser Faktor nur wenig eingeschränkt ist, kann hier mit jedem neuen Aufruf der Klasse ein anderes Netz als optimal ausgewählt werden. Das optimalste Netz richtet sich auch stark nach dem train-dev-test splitting des Datensatzes.

In [ ]:
opt_combination = []
for k in range(2): #30
    if cuda.lower() == 'y':
        l = HP_Layer_Optimizer(layer_range=(8,11),#5,15
                           max_epochs = 4,#10
                           input_start = 52,
                           #activations = ['relu'], 
                           random_activation=True,
                           create_combinations=True, 
                           create_variations = True, 
                           num_variations = 2,#5
                           pure_activations = True,
                           cuda = True)
    else:
        l = HP_Layer_Optimizer(layer_range=(7,12),
                           max_epochs = 4,
                           input_start = 52,
                           #activations = ['relu'], 
                           random_activation=True,
                           create_combinations=True, 
                           create_variations = True, 
                           num_variations = 5,
                           pure_activations = True,
                           cuda = False)
    #l.model_specs_combinations = netze
    l.train_data = train_T
    l.test_data = dev_T
    l.validate_combinations()
    l.get_all_information()
    print(l.opt_combination)
    opt_combination.append(l.opt_combination)
    print("einmal optimieren fertig")

In [35]:
auswertung = pd.DataFrame(columns = ['relu', 'sigmoid', 'tanh', 'layer_sum', 'dropout_loc','mae'])
idx = 0
for combination in opt_combination:
    relu = 0
    sigmoid = 0
    tanh = 0
    layer_sum = 0
    dropout_loc = 0
    mae = 0
    layer_count = 1
    for act, layer in combination.items():
        if act == "mae":
            mae = layer
        else:
            if act.startswith('relu'):
                relu += 1
            elif act.startswith('sigmoid'):
                sigmoid += 1
            elif act.startswith('tanh'):
                tanh += 1
            else:
                pass
            if layer[0] == "dropout":
                dropout_loc = layer_count
        layer_count += 1
    layer_sum = len(list(combination.keys()))-1
    row = [relu, sigmoid, tanh, layer_sum, dropout_loc, mae]
    auswertung.loc[idx, :] = row
    idx += 1
auswertung

,relu,sigmoid,tanh,layer_sum,dropout_loc,mae
0,4,1,2,8,5,1.71098
1,4,1,2,8,4,1.71484


In [36]:
auswertung.to_csv("auswertung_netze.csv",sep = ";")

#### Optimieren der Lernrate und Trainingsepochenanzahl
Paralleles Aufrufen des Optimierers, übergeben der zuvor erzeugten Dev- und Trainingsdatensätze, angeben der Intervalle für die die Hyperparameter getestet werden sollen (Lernrate und Epochenanzahl). Für das dynamische Modell wird das Dictionary übergeben, welches von dem vorhergegangenen Layer Optimizer am besten bewertet wurde und die Variable dynamic auf True gesetzt (default False). Der "normale" HP_Optimizer basiert auf der Klasse Netz().

In [38]:
opt_combination

[{'first': ['linear', 52, 176],
  'sigmoid0': ['linear', 176, 177],
  'tanh1': ['linear', 177, 182],
  'tanh2': ['linear', 182, 197],
  'relu3': ['dropout', 197, 199],
  'relu4': ['linear', 197, 163],
  'relu5': ['linear', 163, 147],
  'relu6': ['linear', 147, 121],
  'mae': 1.7109826589595376},
 {'first': ['linear', 52, 196],
  'relu0': ['linear', 196, 196],
  'tanh1': ['linear', 196, 199],
  'relu2': ['dropout', 199, 199],
  'sigmoid3': ['linear', 199, 200],
  'tanh4': ['linear', 200, 97],
  'relu5': ['linear', 97, 84],
  'relu6': ['linear', 84, 44],
  'mae': 1.7148362235067438}]

In [39]:
dynamische_netze = []
#"korrigieren" der netz dictionarys --> mae aus den keys entfernen 
for netz in opt_combination:
    opt_={}
    for key in netz.keys():
        if key != 'mae':#mae wird noch in dictionary von HP_Optimizer hinzugefügt und muss entfernt werden
            opt_[key] = netz[key]
    dynamische_netze.append(opt_)
    
#statisches netz auf lernrate und trainingsepochen optimieren
if cuda.lower() == 'y':
    h = HP_Optimizer(lr_range = [0.0001,0.00045],step_size = 0.0001, max_epochs=(3,5),cuda = True)
else:
    h = HP_Optimizer(lr_range = [0.0001,0.00045],step_size = 0.0001, max_epochs=(3,5),cuda = False)
    
h.help()
#Zuweisen der Trainingsdaten
h.train_data = train_T
#Zuweisen der Dev Daten als Testdaten
h.test_data = dev_T
#Aufruf der Funktion die verschiedene Kombinationen der Epochen und Lernraten gegeneinander vergleicht
h.validate_combinations()
#Ausgabe der Ergebnisse des Vergleichs aus self.validate_combinations()
print('Statisches Netz():\n')
h.get_all_information()

    
#alle dynamisch erzeugten netze werden auf lernrate und trainingsepocen optimiert
idx = 0
for netz in dynamische_netze:
    #überprüfen ob cuda verwendet werden soll, dann Aufruf des Optimizers für beide Netze
    if cuda.lower() == 'y':
        h_dynamic = HP_Optimizer(lr_range = [0.0001,0.00045],step_size = 0.0001, max_epochs=(3,5),cuda = True, dynamic = True, dyn_combination = netz)
    else:
        h_dynamic = HP_Optimizer(lr_range = [0.0001,0.00045],step_size = 0.0001, max_epochs=(3,5),cuda = False, dynamic = True, dyn_combination = netz)
    h_dynamic.train_data = train_T
    h_dynamic.test_data = dev_T
    h_dynamic.validate_combinations()
    print('\n',25*'=','\n')
    print('Dynamisches Netz:\n')
    h_dynamic.get_all_information()
    
    max_epochs_dyn = h_dynamic.opt_combination['epochen']
    lr_dynamic = h_dynamic.opt_combination['lr']
    mae_dynamic = h_dynamic.opt_combination["mae"]
    netz["epochen"] = max_epochs_dyn
    netz["lr"] = lr_dynamic
    netz["mae"] = mae_dynamic
    dynamische_netze[idx] = netz
    idx += 1

Parameters with defaults:
lr_range --> (0.0001,0.0001),
step_size--> 0.0001,
max_epochs-->(2,2),
opt-->"Adam",
cuda=True
lr_range: Tupel with learnrate range
step_size: float/int for step_size of learnrate
max_epochs: Tupel with number of epochs range
opt: Optimizer (by default Adam)
cuda: True/False if cuda should be used, default = True

Attributes:
set self.train_data as dictionary with races (form: {raceId: race(dataframe)})
set self.test_data as dictionary with races (form: {raceId: race(dataframe)})
self.opt_combination: Dictionary which contains the best combination of the given parameters

Methods:
call self.validate_combination() to compare all combinations
get all information/results with self.get_all_information()


C:\Users\greil\Anaconda3\lib\site-packages\torch\tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Train Epoche: 1 [0/96 (0%)]	Loss: 1.429699
Train Epoche: 1 [1/96 (1%)]	Loss: 10.179125
Train Epoche: 1 [2/96 (2%)]	Loss: 125.302719
Train Epoche: 1 [3/96 (3%)]	Loss: 4.831140
Train Epoche: 1 [4/96 (4%)]	Loss: 103.811295
Train Epoche: 1 [5/96 (5%)]	Loss: 26.846178
Train Epoche: 1 [6/96 (6%)]	Loss: 51.605320
Train Epoche: 1 [7/96 (7%)]	Loss: 173.931610
Train Epoche: 1 [8/96 (8%)]	Loss: 230.364380
Train Epoche: 1 [9/96 (9%)]	Loss: 66.859680
Train Epoche: 1 [10/96 (10%)]	Loss: 38.191833
Train Epoche: 1 [11/96 (11%)]	Loss: 17.467674
Train Epoche: 1 [12/96 (12%)]	Loss: 83.814301
Train Epoche: 1 [13/96 (14%)]	Loss: 200.737656
Train Epoche: 1 [14/96 (15%)]	Loss: 261.413452
Train Epoche: 1 [15/96 (16%)]	Loss: 406.698639
Train Epoche: 1 [16/96 (17%)]	Loss: 148.081970
Train Epoche: 1 [17/96 (18%)]	Loss: 406.024872
Train Epoche: 1 [18/96 (19%)]	Loss: 293.810059
Train Epoche: 1 [19/96 (20%)]	Loss: 9.809864
Train Epoche: 1 [20/96 (21%)]	Loss: 16.886465
Train Epoche: 1 [21/96 (22%)]	Loss: 4.491579
Tr

Train Epoche: 1 [174/96 (181%)]	Loss: 49.366581
Train Epoche: 1 [175/96 (182%)]	Loss: 27.216702
Train Epoche: 1 [176/96 (183%)]	Loss: 0.254574
Train Epoche: 1 [177/96 (184%)]	Loss: 24.879820
Train Epoche: 1 [178/96 (185%)]	Loss: 1.924422
Train Epoche: 1 [179/96 (186%)]	Loss: 332.387817
Train Epoche: 1 [180/96 (188%)]	Loss: 12.615917
Train Epoche: 1 [181/96 (189%)]	Loss: 0.019659
Train Epoche: 1 [182/96 (190%)]	Loss: 141.806305
Train Epoche: 1 [183/96 (191%)]	Loss: 87.629395
Train Epoche: 1 [184/96 (192%)]	Loss: 2.815050
Train Epoche: 1 [185/96 (193%)]	Loss: 104.912834
Train Epoche: 1 [186/96 (194%)]	Loss: 36.499943
Train Epoche: 1 [187/96 (195%)]	Loss: 9.573489
Train Epoche: 1 [188/96 (196%)]	Loss: 303.653320
Train Epoche: 1 [189/96 (197%)]	Loss: 107.832146
Train Epoche: 1 [190/96 (198%)]	Loss: 1.871556
Train Epoche: 1 [191/96 (199%)]	Loss: 3.217425
Train Epoche: 1 [192/96 (200%)]	Loss: 55.175602
Train Epoche: 1 [193/96 (201%)]	Loss: 7.341442
Train Epoche: 1 [194/96 (202%)]	Loss: 376.9

Train Epoche: 1 [345/96 (359%)]	Loss: 5.590157
Train Epoche: 1 [346/96 (360%)]	Loss: 3.347291
Train Epoche: 1 [347/96 (361%)]	Loss: 116.595207
Train Epoche: 1 [348/96 (362%)]	Loss: 22.842134
Train Epoche: 1 [349/96 (364%)]	Loss: 31.682384
Train Epoche: 1 [350/96 (365%)]	Loss: 15.940227
Train Epoche: 1 [351/96 (366%)]	Loss: 267.873352
Train Epoche: 1 [352/96 (367%)]	Loss: 41.934311
Train Epoche: 1 [353/96 (368%)]	Loss: 60.533382
Train Epoche: 1 [354/96 (369%)]	Loss: 369.607605
Train Epoche: 1 [355/96 (370%)]	Loss: 14.508511
Train Epoche: 1 [356/96 (371%)]	Loss: 59.144966
Train Epoche: 1 [357/96 (372%)]	Loss: 0.064524
Train Epoche: 1 [358/96 (373%)]	Loss: 60.249535
Train Epoche: 1 [359/96 (374%)]	Loss: 16.107687
Train Epoche: 1 [360/96 (375%)]	Loss: 10.879889
Train Epoche: 1 [361/96 (376%)]	Loss: 36.573612
Train Epoche: 1 [362/96 (377%)]	Loss: 193.107132
Train Epoche: 1 [363/96 (378%)]	Loss: 12.993873
Train Epoche: 1 [364/96 (379%)]	Loss: 278.151520
Train Epoche: 1 [365/96 (380%)]	Loss: 

...

Train Epoche: 3 [1660/96 (1729%)]	Loss: 2.144148
Train Epoche: 3 [1661/96 (1730%)]	Loss: 1.928220
Train Epoche: 3 [1662/96 (1731%)]	Loss: 7.646629
Train Epoche: 3 [1663/96 (1732%)]	Loss: 98.086220
Train Epoche: 3 [1664/96 (1733%)]	Loss: 4.438141
Train Epoche: 3 [1665/96 (1734%)]	Loss: 14.126840
Train Epoche: 3 [1666/96 (1735%)]	Loss: 3.221323
Train Epoche: 3 [1667/96 (1736%)]	Loss: 24.756708
Train Epoche: 3 [1668/96 (1738%)]	Loss: 1.498486
Train Epoche: 3 [1669/96 (1739%)]	Loss: 14.606057
Train Epoche: 3 [1670/96 (1740%)]	Loss: 40.252605
Train Epoche: 3 [1671/96 (1741%)]	Loss: 3.507969
Train Epoche: 3 [1672/96 (1742%)]	Loss: 7.912868
Train Epoche: 3 [1673/96 (1743%)]	Loss: 3.120852
Train Epoche: 3 [1674/96 (1744%)]	Loss: 0.102164
Train Epoche: 3 [1675/96 (1745%)]	Loss: 1.637023
Train Epoche: 3 [1676/96 (1746%)]	Loss: 5.212111
Train Epoche: 3 [1677/96 (1747%)]	Loss: 9.762126
Train Epoche: 3 [1678/96 (1748%)]	Loss: 0.050732
Train Epoche: 3 [1679/96 (1749%)]	Loss: 0.512933
Train Epoche: 3

Train Epoche: 3 [1826/96 (1902%)]	Loss: 79.887787
Train Epoche: 3 [1827/96 (1903%)]	Loss: 1.340295
Train Epoche: 3 [1828/96 (1904%)]	Loss: 0.245166
Train Epoche: 3 [1829/96 (1905%)]	Loss: 3.940947
Train Epoche: 3 [1830/96 (1906%)]	Loss: 0.060108
Train Epoche: 3 [1831/96 (1907%)]	Loss: 4.035725
Train Epoche: 3 [1832/96 (1908%)]	Loss: 1.880404
Train Epoche: 3 [1833/96 (1909%)]	Loss: 12.241464
Train Epoche: 3 [1834/96 (1910%)]	Loss: 8.753049
Train Epoche: 3 [1835/96 (1911%)]	Loss: 107.794617
Train Epoche: 3 [1836/96 (1912%)]	Loss: 3.314682
Train Epoche: 3 [1837/96 (1914%)]	Loss: 12.215446
Train Epoche: 3 [1838/96 (1915%)]	Loss: 6.214027
Train Epoche: 3 [1839/96 (1916%)]	Loss: 0.002290
Train Epoche: 3 [1840/96 (1917%)]	Loss: 6.450102
Train Epoche: 3 [1841/96 (1918%)]	Loss: 0.940061
Train Epoche: 3 [1842/96 (1919%)]	Loss: 2.450579
Train Epoche: 3 [1843/96 (1920%)]	Loss: 4.390173
Train Epoche: 3 [1844/96 (1921%)]	Loss: 1.313423
Train Epoche: 3 [1845/96 (1922%)]	Loss: 1.161871
Train Epoche: 3

Train Epoche: 3 [1992/96 (2075%)]	Loss: 19.494066
Train Epoche: 3 [1993/96 (2076%)]	Loss: 258.457611
Train Epoche: 3 [1994/96 (2077%)]	Loss: 4.613367
Train Epoche: 3 [1995/96 (2078%)]	Loss: 0.018150
Train Epoche: 3 [1996/96 (2079%)]	Loss: 13.508194
Train Epoche: 3 [1997/96 (2080%)]	Loss: 4.904249
Train Epoche: 3 [1998/96 (2081%)]	Loss: 2.707751
Train Epoche: 3 [1999/96 (2082%)]	Loss: 4.624030
Train Epoche: 3 [2000/96 (2083%)]	Loss: 21.147985
Train Epoche: 3 [2001/96 (2084%)]	Loss: 16.249483
Train Epoche: 3 [2002/96 (2085%)]	Loss: 18.731476
Train Epoche: 3 [2003/96 (2086%)]	Loss: 3.203917
Train Epoche: 3 [2004/96 (2088%)]	Loss: 16.262812
Train Epoche: 3 [2005/96 (2089%)]	Loss: 11.355877
Train Epoche: 3 [2006/96 (2090%)]	Loss: 16.440195
Train Epoche: 3 [2007/96 (2091%)]	Loss: 7.092907
Train Epoche: 3 [2008/96 (2092%)]	Loss: 10.052106
Train Epoche: 3 [2009/96 (2093%)]	Loss: 16.858774
Train Epoche: 3 [2010/96 (2094%)]	Loss: 11.175496
Train Epoche: 3 [2011/96 (2095%)]	Loss: 2.488054
Train E

C:\Users\greil\Anaconda3\lib\site-packages\torch\tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
C:\Users\greil\Anaconda3\lib\site-packages\torch\tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
C:\Users\greil\Anaconda3\lib\site-packages\torch\tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
C:\Users\greil\Anaconda3\lib\site-packages\torch\tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
C:\Users\greil\Anaconda3\lib\site-packages\torch\tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
C:\Users\greil\Anaconda3\lib\site-packages\torch\tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
C:\Users\greil\Anaconda3\lib



Dynamisches Netz:

Chosen Model: NetzDynamic(
  (layers): ModuleList(
    (0): Linear(in_features=52, out_features=196, bias=True)
    (1): Linear(in_features=196, out_features=196, bias=True)
    (2): Linear(in_features=196, out_features=199, bias=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=199, out_features=200, bias=True)
    (5): Linear(in_features=200, out_features=97, bias=True)
    (6): Linear(in_features=97, out_features=84, bias=True)
    (7): Linear(in_features=84, out_features=44, bias=True)
  )
)
Learningrate Range: [0.0001, 0.00045]
Maximum Epochs: (3, 5)
Chosen Optimizer: Adam
Result Encoding: {7161: {'lr': 0.0001, 'epochen': 3}, 3954: {'lr': 0.00045, 'epochen': 3}, 7206: {'lr': 0.0001, 'epochen': 4, 'mae': 1.7148362235067438}, 1853: {'lr': 0.00045, 'epochen': 4}}
Results: {7161: 1.7302504816955684, 3954: 2.458574181117534, 7206: 1.7148362235067438, 1853: 2.1001926782273603}
Optimale Kombination: {'lr': 0.0001, 'epochen': 4, 'mae': 1.7148362

In [41]:
dynamische_netze

[{'first': ['linear', 52, 176],
  'sigmoid0': ['linear', 176, 177],
  'tanh1': ['linear', 177, 182],
  'tanh2': ['linear', 182, 197],
  'relu3': ['dropout', 197, 199],
  'relu4': ['linear', 197, 163],
  'relu5': ['linear', 163, 147],
  'relu6': ['linear', 147, 121],
  'epochen': 4,
  'lr': 0.00045,
  'mae': 1.5183044315992293},
 {'first': ['linear', 52, 196],
  'relu0': ['linear', 196, 196],
  'tanh1': ['linear', 196, 199],
  'relu2': ['dropout', 199, 199],
  'sigmoid3': ['linear', 199, 200],
  'tanh4': ['linear', 200, 97],
  'relu5': ['linear', 97, 84],
  'relu6': ['linear', 84, 44],
  'epochen': 4,
  'lr': 0.0001,
  'mae': 1.7148362235067438}]

#### Testen der erzeugten Netze
Es werden die als optimal in den gegebenen Intervallen bestimmten Werte aus dem Optimierer genommen und zwei neue Netze mit ihren jeweiligen Parametern für Trainingsepochen und Lernrate trainiert (dies geschieht für das statische und das dynamische Netz). Zum Schluss werden beide Netze auf den Testdaten getestet und die Ergebnisse des Testlaufes ausgegeben.

In [42]:
max_epochs = h.opt_combination['epochen']
#max_epochs_dyn = h_dynamic.opt_combination['epochen']
lr = h.opt_combination['lr']
#lr_dynamic = h_dynamic.opt_combination['lr']
model = Netz()

opt={}
for key in opt_combination.keys():
    if key != 'mae':#mae wird noch in dictionary von HP_Optimizer hinzugefügt und muss entfernt werden
        opt[key] = opt_combination[key]
model_dynamic = NetzDynamic(opt)
if cuda.lower() == 'y':
    model.cuda() 
    model_dynamic.cuda()

    
optimizer = optim.Adam(model.parameters(), lr = lr)     
optimizer_d = optim.Adam(model_dynamic.parameters(), lr = lr_dynamic)   

def train_cuda(epoch):
    model.train()
    batch_id = 0
    for key in train_T.keys():
        
        for data, target in train_T[key]:
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0).cuda()
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            optimizer.zero_grad()
            out = model(data)
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
def train_(epoch):
    model.train()
    batch_id = 0
    for key in train_T.keys():
        for data, target in train_T[key]:
            #data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)#.cuda()
            shape = target.size()[1]
            target = target.resize(shape,1)#.cuda()
            optimizer.zero_grad()
            out = model(data)
            #print("Out: ", out, out.size())
            #print("Target: ", target, target.size())
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
            
def train_cuda_dynamic(epoch):
    model_dynamic.train()
    batch_id = 0
    for key in train_T.keys():
        
        for data, target in train_T[key]:
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0).cuda()
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            optimizer_d.zero_grad()
            out = model_dynamic(data)
            #print("Out: ", out, out.size())
            #print("Target: ", target, target.size())
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer_d.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
def train_dynamic(epoch):
    model_dynamic.train()
    batch_id = 0
    for key in train_T.keys():
        for data, target in train_T[key]:
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1)
            optimizer_d.zero_grad()
            out = model_dynamic(data)
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer_d.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
if cuda.lower() == 'y':
    for epoch in range(1,max_epochs):
        train_cuda(epoch) 
    for epoch in range(1,max_epochs_dyn):
        train_cuda_dynamic(epoch) 
else:
    for epoch in range(1,max_epochs):
        train_(epoch)  
    for epoch in range(1,max_epochs_dyn):
        train_dynamic(epoch)  

        
def test_times_cuda(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        help_dict = {}
        for data, target in test_final[key]:
            model.eval()
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            out = model(data).cpu()
            out = out.detach().numpy()
            target = target.cpu()
            target = target.detach().numpy()
            total += abs(out - target[0][0])
            help_dict[target[0][0]] = out
            count+=1
        result_dict[key] = help_dict
        
    return result_dict

def test_times(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        help_dict = {}
        for data, target in test_final[key]:
            model.eval()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1)
            out = model(data)
            out = out.detach().numpy()
            target = target.detach().numpy()
            total += abs(out - target[0][0])
            help_dict[target[0][0]] = out
            count+=1
        result_dict[key] = help_dict
        
    return result_dict

def test_times_cuda_dynamic(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        help_dict = {}
        for data, target in test_final[key]:
            model_dynamic.eval()
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            out = model_dynamic(data).cpu()
            out = out.detach().numpy()
            target = target.cpu()
            target = target.detach().numpy()
            total += abs(out - target[0][0])
            help_dict[target[0][0]] = out
            count+=1
        result_dict[key] = help_dict
        
    return result_dict

def test_times_dynamic(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        #print(key)
        help_dict = {}
        for data, target in test_final[key]:
            model_dynamic.eval()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1)
            out = model_dynamic(data)
            out = out.detach().numpy()
            target = target.detach().numpy()
            total += abs(out - target[0][0])
            help_dict[target[0][0]] = out
            count+=1
        result_dict[key] = help_dict
        
    return result_dict


if cuda.lower() =='y':
    total_results = test_times_cuda(test_T)
    total_results_dynamic = test_times_cuda_dynamic(test_T)
else:
    total_results = test_times(test_T)
    total_results_dynamic = test_times_dynamic(test_T)


AttributeError: 'list' object has no attribute 'keys'

In [43]:
result_dfs = {}
for raceId, dict in total_results.items():    
    #Auswerten der Vorhersagen aus den Outputdaten des Modells
    A = [x[0][0] for x in list((dict.values()))]
    A_dyn = [x[0][0] for x in list((total_results_dynamic[raceId].values()))]
    y = list(dict.keys())
    y_dyn = list(total_results_dynamic[raceId].keys())
    t = pd.DataFrame(columns = ['target', 'prediction'])
    t_dyn = pd.DataFrame(columns = ['target', 'prediction'])
    t['target'] = y
    t['prediction'] = A
    t_dyn['target'] = y_dyn
    t_dyn['prediction'] = A_dyn
    #print("raceId", raceId)
    temp = sliced_races[raceId]
    #name = split_by_race[raceId].where()
    #sortieren des DataFrames nach den vorhergesagten Positionen, aufsteigend
    end = sqldf.sqldf('''select * from t order by prediction ASC''')
    end.reset_index(inplace = True)
    end_dyn = sqldf.sqldf('''select * from t_dyn order by prediction ASC''')
    end_dyn.reset_index(inplace = True)
    #Da DF nun nach prediction aufsteigend sortiert ist, kann veränderter Index als predictete Position gesetzt werden
    end.rename(columns = {'index': 'predicted_position'},inplace = True)
    end_dyn.rename(columns = {'index': 'predicted_position'},inplace = True)
    #zur Übersichtlichkeit wird nun der endgültige DF nach den richtigen Positionen (target) sortiert (aufsteigend)
    end = sqldf.sqldf('''select * from end order by target ASC''')
    end_dyn = sqldf.sqldf('''select * from end_dyn order by target ASC''')
    end['predicted_position'] = end['predicted_position']+1
    end_dyn['predicted_position'] = end_dyn['predicted_position']+1
    #pred_name = sqldf.sqldf("""select 
    #            distinct
    #            d.driver_fullname as name,
    #            t.predicted_position as pred,
    #            d.podium_position as target_position
    #            from 
    #            end t inner join temp d
    #            on t.predicted_position = d.podium_position""")
    end["driver_pred"] = 0
    end["driver_target"] = 0
    end_dyn["driver_pred"] = 0
    end_dyn["driver_target"] = 0
    for pos in temp.podium_position.unique():
        
        name = temp.where(temp.podium_position == pos).dropna(how = "all")
        #print(name)
        name_ = name["driver_fullname"][list(name.index)[0]]
        idx_target = end.where(end.target == pos).dropna(how = "all").index
        idx_pred = end.where(end.predicted_position == pos).dropna(how = "all").index
        end.loc[idx_target,"driver_target"] = name_
        end.loc[idx_pred,"driver_pred"] = name_
        
        idx_target = end_dyn.where(end_dyn.target == pos).dropna(how = "all").index
        idx_pred = end_dyn.where(end_dyn.predicted_position == pos).dropna(how = "all").index
        end_dyn.loc[idx_target,"driver_target"] = name_
        end_dyn.loc[idx_pred,"driver_pred"] = name_
        
    #umstellen der Spaltenreihenfolge
    '''end = end[['target', 'predicted_position', 'prediction',"driver_target", "driver_pred"]]
    end_dyn = end_dyn[['target', 'predicted_position', 'prediction',"driver_target", "driver_pred"]]'''
    end = end[['target', 'predicted_position', 'prediction']]
    end_dyn = end_dyn[['target', 'predicted_position', 'prediction']]
    result_dfs[raceId] = end
    r = str(raceId)+'_dynamic'
    result_dfs[r] = end_dyn
    
for key, value in result_dfs.items():
    print('RaceId:',key)
    print(value.head(10),'\n')

NameError: name 'total_results' is not defined